**Author:** Andy Tzanidakis, Neven Caplar and the LINCC Frameworks team \
**Last updated**: November 07, 2024

## Overview

In this notebook we will learn how to:

- Query and filter catalog data
- Compute time-series features for LSDB catalogs using `nested-dask`
- Plot light curves and periodograms

In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Standard library imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import rcParams
from matplotlib.colors import LogNorm
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
import dask
from dask.distributed import Client

# Local library-specific imports
import lsdb
from lsdb.core.search import BoxSearch, ConeSearch, PolygonSearch
from hats.inspection import plot_pixels
from hats.io.file_io import read_parquet_metadata

# Astropy
from astropy.timeseries import LombScargle

# light-curve package for feature extraction
import light_curve as licu

# Jupyter-specific settings and magic commands
%matplotlib inline
%config InlineBackend.figure_format = "retina"

# Configuration settings
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')
mpl.rcParams['axes.linewidth'] = 2

print(f'Version of lsdb is {lsdb.__version__}')

Version of lsdb is 0.4.2


In [3]:
catalogs_dir = "https://data.lsdb.io/hats"

# Gaia
gaia_path = f"{catalogs_dir}/gaia_dr3/gaia"
gaia_margin_path = f"{catalogs_dir}/gaia_dr3/gaia_10arcs"

# ZTF
ztf_object_path = f"{catalogs_dir}/ztf_dr14/ztf_object"
ztf_source_path = f"{catalogs_dir}/ztf_dr14/ztf_source"
ztf_object_margin_path = f"{catalogs_dir}/ztf_dr14/ztf_object_10arcs"

## Initialize Dask Client

Before we start building our workflow, let's initialize our Dask Client. The machine we're using has 16 cores and 64GiB of RAM so we'll decide to make good use of resources and go with 10 workers, each with 6 GiB of memory and 1 thread.

In [4]:
from dask.distributed import Client
client = Client(n_workers=10, memory_limit="6GiB", threads_per_worker=1)

## Get Gaia and ZTF

Let's read Gaia and ZTF (both lazily) as demonstrated in the first notebook.

In [5]:
%%time

# Load Gaia object table (with margins!)
gaia = lsdb.read_hats(gaia_path, columns=['ra', 'dec', 'parallax'], margin_cache=gaia_margin_path)

# Define a cone region of interest for ZTF
cone_search = ConeSearch(ra=132.8460000, dec=+11.8140000, radius_arcsec=5_000)

# Load ZTF object table
ztf = lsdb.read_hats(ztf_object_path, columns=['ra', 'dec', 'ps1_objid'], search_filter=cone_search)

# Load ZTF DR14 sources
ztf_sources = lsdb.read_hats(ztf_source_path, columns=['ra', 'dec', 'mjd', 'mag', 'magerr', 'band', 'ps1_objid', 'catflags'], search_filter=cone_search)

KeyboardInterrupt: 

## Create a sample object catalog

To create our sample let's crossmatch our ZTF region with Gaia with a 3-arcsecond radius.

In [6]:
_sample = ztf.crossmatch(gaia, radius_arcsec=3)

NameError: name 'ztf' is not defined

We can have a look at the object matches because this sample is relatively small and it fits in memory! 

In [ ]:
%%time
_sample_computed = _sample.compute()
_sample_computed

In [ ]:
print (f"Number of objects in crossmatch: {len(_sample_computed)}")

We selected a very small region of the sky - 5000 arcsec - and, as a result, our objects are contained in just a handful of pixels. In the following mollview of the pixel map, the gray color means that our sample catalog does not have coverage for that region in space.

In [ ]:
_sample.plot_coverage(
    center=SkyCoord(ra=132.8460000, dec=+11.8140000, unit='deg'),
    fov=(Angle("20deg"), Angle("20deg")),
)
fig, ax = _sample.plot_pixels(
    center=SkyCoord(ra=132.8460000, dec=+11.8140000, unit='deg'),
    fov=(Angle("20deg"), Angle("20deg")),
    edgecolor='grey',
)

## Fetch ZTF light curves for the samples

We will now perform a join operation between our sample objects and their respective ZTF sources. We will use the PanSTARRS object ID, as this is the index that both catalogs share. This is where `nested-dask` features come in handy. Under the hood, this `join_nested` call packs the time-domain data of each object as an additional column, which we decided to conveniently named `sources`.

In [ ]:
_sources = _sample.join_nested(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid", nested_column_name="sources")
_sources

Note that we also get a warning stating that we are not using a margin catalog, as previously discussed in the first notebook.

## Filter on r-band

Our workflow will only operate on r-band data. We can apply the necessary cuts to our data to make sure that only light curves that have at least 10 observations in r-band will be considered!

In [ ]:
# Query for valid observations in r-band
r_band = _sources.query(
    "sources.band == 'r'"  # r-band only
    " and sources.catflags == 0"  # good observational conditions
    " and sources.mag < 99"  # valid magnitude
    " and 0 < sources.magerr < 99"  # valid magnitude error
)
# Calculate the number of observations
r_band = r_band.reduce(lambda mjd: {"nobs": mjd.size}, "sources.mjd", meta={"nobs": int}, append_columns=True)
r_band = r_band.query("nobs > 10")

In [ ]:
r_band

In [ ]:
%%time
# Cache the data in memory - we will use it later again
r_band._ddf = r_band._ddf.persist()
# Convert lsdb.Catalog to pandas.DataFrame
r_band.compute()

## Extract features from light curves

We will now write custom analysis functions to compute features (statistics) on each light curve. Some of these timeseries features (e.g. Von Neumann, excess variance) characterize the amount of outliers in the timeseries. They have been heavily used in literature, for example, to find variable stars.

In [ ]:
extractor = licu.Extractor(
    licu.Periodogram(
        peaks=1,
        max_freq_factor=10.0,
        resolution=50.0,
        # nyquist='median',
        # fast=True,
    ),  # Would give two features: peak period and signa-to-noise ratio of the peak
    licu.WeightedMean(),  # Mean magnitude
    licu.Eta(),  # Von Neumann's eta statistics
    licu.ExcessVariance(),  # Excess variance statistics
    licu.Amplitude(),  # 0.5 * [max(mag) - min(mag)]
)


# light-curve package requires all arrays to be the same dtype.
# It also requires the time array to be ordered and to have no duplicates.
def calc_features(mjd, mag, magerr, **kwargs):
    # We offset date, so we still would have <1 second precision
    t = np.asarray(mjd - 60000, dtype=np.float32)
    _, sort_index = np.unique(t, return_index=True)
    features = extractor(
        t[sort_index],
        mag[sort_index],
        magerr[sort_index],
        **kwargs,
    )
    # Return the features as a dictionary
    return dict(zip(extractor.names, features))


features = r_band.reduce(
    calc_features,
    "sources.mjd",
    "sources.mag",
    "sources.magerr",
    meta={name: np.float32 for name in extractor.names},
    append_columns=True,
)
features

We can now apply the feature calculator function `calc_features` to each light curve.

Let's compute the result (bring it into memory). This will take a minute or two and display some expected runtime warnings.

In [ ]:
%%time
features_computed = features.compute()

Let's inspect the results table and make some plots of our time series features.

In [ ]:
# We have the results!
features_computed

In this figure we demonstrate the periods corresponding to highest power in the Lomb Scargle periodogram. Immediately, we can see a few overdensities of period, that are likely originating from the ZTF aliasing sampling cadence. Between those aliasing periods, those with high SNR score are possible periodic candidates that will require further investigation.

In [ ]:
lg_period_bins = np.linspace(-0.5, 2.5, 101)
lg_snr_bins = np.linspace(0.5, 1.5, 101)
plt.hist2d(
    np.log10(features_computed["period_0"]),
    np.log10(features_computed["period_s_to_n_0"]),
    bins=[lg_period_bins, lg_snr_bins],
    norm=LogNorm(),
)
plt.colorbar(label="count")
plt.xlabel('lg(Period/day)')
plt.ylabel('lg(Periodogram Peak S/N)')
plt.minorticks_on()

Below is the ZTF periodogram where also observe the aliasing sampling periods. For more information about periodograms please have a look at the [references](#References) section of this notebook.

![ZTF periodogram](https://raw.githubusercontent.com/lincc-frameworks/MWGaia-DN_Introductory_School/main/assets/ztf-periodogram.jpg)

## Plot light curve

We can now find a ZTF object with a high period SNR and that is bright, and plot the ZTF-r band light curve.

In [ ]:
# Select good SNR candidate
# Filter out bogus periods
good_periods = features_computed.query(
    "period_0 < 400"  # large periods are likely to be bogus, e.g. about of duration of light curve
    " and not(29 < period_0 < 30)"  # lunar month
    " and not(0.95 < period_0 < 1.05)"  # day
)
# Select bright stars with highest S/N of the peridogram peak
top5 = good_periods.sort_values('period_s_to_n_0').tail(5)
top5

In [ ]:
# Select a light curve with high SNR
i = 4
best_period = top5['period_0'].iloc[i]  # (in days) best period from the above table
lc = top5['sources'].iloc[i]
lc

When we fold the light curve on the correct period the scatter between the points is minimized, and therefore, the observations line up.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 5))

ax[0].errorbar(
    lc["mjd"],
    lc["mag"],
    yerr=lc["magerr"], 
    fmt='.',
    color='indianred'
)

ax[1].errorbar((lc["mjd"].to_numpy()%best_period)/best_period, lc["mag"],
                yerr=lc["magerr"], fmt='.', color='#AA4A44')
ax[1].set_xlabel('Phase')
ax[0].set_xlabel('Time [MJD]')
ax[1].tick_params(axis='x', which='both', bottom=True, top=True, direction='in')
ax[0].tick_params(axis='x', which='both', bottom=True, top=True, direction='in')
ax[1].tick_params(axis='y', which='both', bottom=True, top=True, direction='in')
ax[0].tick_params(axis='y', which='both', bottom=True, top=True, direction='in')
ax[0].set_ylabel('Magnitude')
ax[0].invert_yaxis()
ax[1].invert_yaxis()
ax[0].set_title("Observed", weight='bold')
ax[1].set_title(f"Phase-folded, P={best_period:.2f}d", weight='bold')

plt.tight_layout()

## Extra topics

### Existing feature extractors

In this notebook we demonstrated how you can define and use your own custom functions to operate on light curves. There are also third-party packages that extract common features so that you don't have to implement these operations yourself (e.g. [`light-curve`](https://github.com/light-curve/light-curve-python) package). For more information please read the following [tutorial](https://nested-dask.readthedocs.io/en/latest/tutorials/work_with_lsdb.html#Extract-features-from-ZTF-light-curves).

### Example of very large analysis

This is an example of a similar analysis as above, but on the entire ZTF DR14 dataset, extracting periods from ~billion light curves. The computation took 90 minutes on the Pittsburgh Supercomputer Center resourcees. We used a 256GB manager node and 50 × 32GB workers (3 threads per worker) allocated as a 50 SLURM jobs.

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('04lGbtNTzZw', width=800, height=450)

#### Close the DASK client

In [ ]:
# Release allocated resources!
client.close()

### References

[1]: https://iopscience.iop.org/article/10.3847/1538-4365/aab766 \
[2]: https://academic.oup.com/mnras/article/505/2/2954/6284767 \
[3]: https://www.astroml.org/gatspy/periodic/lomb_scargle_multiband.html \
[4]: Science with ZTF: https://iopscience.iop.org/article/10.1088/1538-3873/ab006c/pdf \
[5]: https://www.youtube.com/watch?v=2EwtD3Nhazs